In [66]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [67]:
!pip install pytorch-pretrained-bert pytorch-nlp

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [76]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla K80'

In [77]:
# Upload the train file from your local drive
from google.colab import files
uploaded = files.upload()

Saving Tweets.csv to Tweets (1).csv


In [78]:
df=pd.read_csv('Tweets.csv')
df.head()


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [79]:
labels=pd.get_dummies(df['airline_sentiment']).values
label=[]
for term in labels:
  label.append(np.argmax(term))
print(label)

[1, 2, 1, 0, 0, 0, 2, 1, 2, 2, 1, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 2, 1, 2, 2, 1, 0, 2, 0, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 0, 2, 2, 1, 1, 1, 0, 1, 1, 2, 1, 0, 0, 2, 0, 1, 1, 1, 0, 2, 2, 1, 1, 0, 1, 0, 2, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 2, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 1, 0, 2, 1, 1, 0, 0, 1, 0, 0, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 0, 0, 2, 2, 0, 0, 1, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 0, 2, 2, 1, 0, 0, 1, 2, 1, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 1, 2, 2, 2, 2, 1, 2, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 2, 0, 1, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 1, 1, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 1, 2, 1, 1, 1, 2, 0, 1, 1, 0, 0, 2, 0, 0, 1, 1, 2, 1, 1, 2, 2, 0, 1, 2, 2, 1, 1, 0, 2, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 2, 1, 1, 1, 1, 0, 1, 1, 0, 1, 2, 0, 0, 2, 2, 2, 1, 1, 2, 1, 2, 0, 1, 0, 0, 1, 1, 0, 0, 1, 2, 2, 1, 1, 1, 1, 0, 1, 2, 2, 

In [0]:
import re
features=df.iloc[:,10].values
processed_features=[]

for sentence in range(0, len(features)):
    # Remove all the special characters
    processed_feature = re.sub(r'\W', ' ', str(features[sentence]))

    # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'
    processed_feature = re.sub(r'^b\s+', '', processed_feature)

    # Converting to Lowercase
    processed_feature = processed_feature.lower()
    processed_features.append(processed_feature)

df_bert=pd.DataFrame({
    'id':range(len(df)),
    'label':label,
    'alpha':['a']*df.shape[0],
    'text': processed_features
    
})

In [81]:
df_bert.head()

,id,label,alpha,text
0,0,1,a,virginamerica what dhepburn said
1,1,2,a,virginamerica plus you ve added commercials t...
2,2,1,a,virginamerica didn today must mean need to ta...
3,3,0,a,virginamerica it really aggressive to blast o...
4,4,0,a,virginamerica and it a really big bad thing a...


In [0]:
df_bert.to_csv('in_domain_train.tsv', sep='\t', index=False, header=False)

In [0]:
df = pd.read_csv("in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

In [84]:
df.shape

(14640, 4)

In [85]:
df.head(10)

,sentence_source,label,label_notes,sentence
0,0,1,a,virginamerica what dhepburn said
1,1,2,a,virginamerica plus you ve added commercials t...
2,2,1,a,virginamerica didn today must mean need to ta...
3,3,0,a,virginamerica it really aggressive to blast o...
4,4,0,a,virginamerica and it a really big bad thing a...
5,5,0,a,virginamerica seriously would pay 30 flight f...
6,6,2,a,virginamerica yes nearly every time fly vx th...
7,7,1,a,virginamerica really missed prime opportunity...
8,8,2,a,virginamerica well didn but now do d
9,9,2,a,virginamerica it was amazing and arrived an h...


In [0]:
# Create sentence and label lists
sentences = df.sentence.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = df.label.values

In [87]:
sentences[1]

'[CLS]  virginamerica plus you ve added commercials to the experience tacky  [SEP]'

In [88]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[1])

Tokenize the first sentence:
['[CLS]', 'virgin', '##ame', '##rica', 'plus', 'you', 've', 'added', 'commercials', 'to', 'the', 'experience', 'tack', '##y', '[SEP]']


In [0]:
# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 128
# Pad our input tokens
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [0]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq] #bigger than one means this data, or this is a padding
  attention_masks.append(seq_mask)

In [0]:
# Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [0]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [0]:
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [95]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediat

In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]



In [97]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

t_total value of -1 results in schedule not being applied


In [0]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [99]:

train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
  
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Train loss: 0.5209079966383073


Epoch:  25%|██▌       | 1/4 [10:31<31:34, 631.44s/it]

Validation Accuracy: 0.8541666666666666
Train loss: 0.28965830059667813


Epoch:  50%|█████     | 2/4 [21:03<21:03, 631.66s/it]

Validation Accuracy: 0.8521286231884058
Train loss: 0.1526224273392587


Epoch:  75%|███████▌  | 3/4 [31:35<10:31, 631.68s/it]

Validation Accuracy: 0.8405797101449275
Train loss: 0.08339338431822009


Epoch: 100%|██████████| 4/4 [42:08<00:00, 631.98s/it]

Validation Accuracy: 0.8455615942028986


In [28]:
b_labels

tensor([1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1],
       device='cuda:0')

In [33]:
# Upload the test file from your local drive
from google.colab import files
uploaded = files.upload()

Saving out_of_domain_dev.tsv to out_of_domain_dev.tsv


In [0]:
df = pd.read_csv("out_of_domain_dev.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

# Create sentence and label lists
sentences = df.sentence.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = df.label.values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


MAX_LEN = 128
# Pad our input tokens
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
  
batch_size = 32  


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [0]:
# Prediction on test set

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [37]:
predictions


[array([[-2.2622886 ,  2.5993645 ],
        [-2.3717115 ,  2.8354952 ],
        [-2.3015335 ,  2.6665432 ],
        [-2.4301915 ,  2.9339275 ],
        [-2.1601715 ,  2.51347   ],
        [-2.9515092 ,  3.5902207 ],
        [-1.4227457 ,  1.6033558 ],
        [-1.5927999 ,  1.8042362 ],
        [-3.192561  ,  3.6505125 ],
        [-2.7694087 ,  3.1830168 ],
        [-0.89799523,  1.1368886 ],
        [-1.3607005 ,  1.6509771 ],
        [-1.3652953 ,  1.3680084 ],
        [-1.9500437 ,  2.30296   ],
        [-1.1216666 ,  1.3174176 ],
        [-0.42140293,  0.59089285],
        [-1.5527184 ,  1.731818  ],
        [ 2.0201726 , -2.1920152 ],
        [-1.010224  ,  1.2788427 ],
        [-1.2725444 ,  1.538441  ],
        [-1.010224  ,  1.2788427 ],
        [-1.2633493 ,  1.4467535 ],
        [-0.70070815,  0.94094265],
        [-0.83291423,  1.1118445 ],
        [-0.42193073,  0.60106385],
        [-1.3481379 ,  1.5421296 ],
        [-0.8258196 ,  1.0281365 ],
        [-2.0706937 ,  2.344

In [38]:
predictions[0]

array([[-2.2622886 ,  2.5993645 ],
       [-2.3717115 ,  2.8354952 ],
       [-2.3015335 ,  2.6665432 ],
       [-2.4301915 ,  2.9339275 ],
       [-2.1601715 ,  2.51347   ],
       [-2.9515092 ,  3.5902207 ],
       [-1.4227457 ,  1.6033558 ],
       [-1.5927999 ,  1.8042362 ],
       [-3.192561  ,  3.6505125 ],
       [-2.7694087 ,  3.1830168 ],
       [-0.89799523,  1.1368886 ],
       [-1.3607005 ,  1.6509771 ],
       [-1.3652953 ,  1.3680084 ],
       [-1.9500437 ,  2.30296   ],
       [-1.1216666 ,  1.3174176 ],
       [-0.42140293,  0.59089285],
       [-1.5527184 ,  1.731818  ],
       [ 2.0201726 , -2.1920152 ],
       [-1.010224  ,  1.2788427 ],
       [-1.2725444 ,  1.538441  ],
       [-1.010224  ,  1.2788427 ],
       [-1.2633493 ,  1.4467535 ],
       [-0.70070815,  0.94094265],
       [-0.83291423,  1.1118445 ],
       [-0.42193073,  0.60106385],
       [-1.3481379 ,  1.5421296 ],
       [-0.8258196 ,  1.0281365 ],
       [-2.0706937 ,  2.3443427 ],
       [-2.017637  ,

In [39]:
# Import and evaluate each test batch using Matthew's correlation coefficient
from sklearn.metrics import matthews_corrcoef
matthews_set = []

for i in range(len(true_labels)):
  matthews = matthews_corrcoef(true_labels[i],
                 np.argmax(predictions[i], axis=1).flatten())
  matthews_set.append(matthews)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [0]:
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

In [1]:
matthews_corrcoef(flat_true_labels, flat_predictions)

NameError: ignored